In [ ]:
# Install Hugging Face libraries
!pip install transformers[torch] datasets accelerate evaluate torch -U

import os
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config, GPT2ForSequenceClassification, TrainingArguments, Trainer
from evaluate import load
import torch
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from collections import defaultdict, Counter
import numpy as np
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers


In [ ]:
# Disable WANDB login requirement
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Mount Google Drive (for Data Storage)
"""from google.colab import drive
drive.mount('/content/drive')"""

Mounted at /content/drive


In [ ]:
def parse_json_with_multiple_grades(json_file):
    """
    Extracts grade levels and full statements from a JSON file.
    Handles both 'educationLevel' and 'gradeLevels' fields for grade data.
    Creates a separate entry for each grade associated with a statement.
    """
    with open(json_file, "r") as f:
        data = json.load(f)

    curriculum_data = []

    # Handle case where the JSON root is a list or a dictionary
    items = data if isinstance(data, list) else data.get("CFItems", [])

    for item in items:
        # Try both 'educationLevel' and 'gradeLevels'
        grade_levels = item.get("educationLevel", []) or item.get("gradeLevels", [])
        statement = item.get("fullStatement", "").strip() or item.get("statement", "").strip()

        if grade_levels and statement:
            # Create a separate entry for each grade
            for level in grade_levels:
                curriculum_data.append({
                    "text": statement,
                    "label": level  # Keep grade as a string for now
                })

    return curriculum_data


# Combine data from all JSON files
all_data = []
json_file_paths = [
    "NLP_Final_Project/State_JSON/Alabama_Standards.json",
    "NLP_Final_Project/State_JSON/Chicago_Standards.json",
    "NLP_Final_Project/State_JSON/Columbia_County_Standards.json",
    "NLP_Final_Project/State_JSON/Florida_Standards.json",
    "NLP_Final_Project/State_JSON/Georgia_Standards.json",
    "NLP_Final_Project/State_JSON/Indiana_Standards.json",
    "NLP_Final_Project/State_JSON/Maine_Standards.json",
    "NLP_Final_Project/State_JSON/Nebraska_Standards.json",
    "NLP_Final_Project/State_JSON/North_Dakota_Standards.json",
    "NLP_Final_Project/State_JSON/Oklahoma_Standards.json",
    "NLP_Final_Project/State_JSON/Oregon_Standards.json",
    "NLP_Final_Project/State_JSON/Rhode_Island_Standards.json",
    "NLP_Final_Project/State_JSON/South_Carolina_Standards.json",
    "NLP_Final_Project/State_JSON/Texas_Standards.json",
    "NLP_Final_Project/State_JSON/Virginia_Standards.json",
    "NLP_Final_Project/State_JSON/Wisconsin_Standards.json",
    "NLP_Final_Project/State_JSON/literacy-0.8.0.json",
    "NLP_Final_Project/State_JSON/BEST_Standards.json",
    "NLP_Final_Project/State_JSON/Board_Education_Standards.json",
    "NLP_Final_Project/State_JSON/Gwinnett_County_Standards.json",
    "NLP_Final_Project/State_JSON/Idaho_Standards.json",
    "NLP_Final_Project/State_JSON/Pennsylvania_Standards.json",
    "NLP_Final_Project/State_JSON/Washington_DC_Standards.json"
]
for file_path in json_file_paths:
    all_data.extend(parse_json_with_multiple_grades(file_path))

print(f"Total samples: {len(all_data)}")
print("Sample entry:", all_data[:3])

# Map grade levels to numeric labels
grade_mapping = {
    "KG": 0,   # Kindergarten
    "01": 1,   # 1st Grade
    "02": 2,   # 2nd Grade
    "03": 3,   # 3rd Grade
    "04": 4,   # 4th Grade
    "05": 5,   # 5th Grade
    "06": 6,   # 6th Grade
    "07": 7,   # 7th Grade
    "08": 8,   # 8th Grade
    "09": 9,   # 9th Grade
    "10": 10,  # 10th Grade
    "11": 11,  # 11th Grade
    "12": 12   # 12th Grade
}

grade_cluster_mapping = {
    0: 0,  # KG -> Early Childhood
    1: 0,  # 1st Grade -> Early Childhood
    2: 0,  # 2nd Grade -> Early Childhood
    3: 0,  # 3rd Grade -> Elementary
    4: 0,  # 4th Grade -> Elementary
    5: 1,  # 5th Grade -> Elementary
    6: 1,  # 6th Grade -> Middle School
    7: 1,  # 7th Grade -> Middle School
    8: 1,  # 8th Grade -> Middle School
    9: 2,  # 9th Grade -> High School
    10: 2,  # 10th Grade -> High School
    11: 2,  # 11th Grade -> High School
    12: 2   # 12th Grade -> High School
}


# Apply the mapping
for entry in all_data:
    entry["label"] = grade_mapping.get(entry["label"], -1)  # Default to -1 if unknown

# Apply the cluster mapping
for entry in all_data:
    entry["label"] = grade_cluster_mapping.get(entry["label"], -1)  # Default to -1 if not found

# Filter out any invalid entries
all_data = [entry for entry in all_data if entry["label"] != -1]
print(f"Filtered data samples: {len(all_data)}")

Total samples: 41797
Sample entry: [{'text': 'Kindergarten', 'label': 'KG'}, {'text': 'Recurring Standards', 'label': 'KG'}, {'text': 'Utilize active listening skills during discussion and conversation in pairs, small groups, or whole-class settings, following agreed-upon rules for participation.', 'label': 'KG'}]
Filtered data samples: 41710


In [ ]:
# Convert all_data into the correct format for Dataset.from_dict
all_data_dict = {
    "text": [entry["text"] for entry in all_data],
    "label": [entry["label"] for entry in all_data]
}

dataset = Dataset.from_dict(all_data_dict)

# Split into train, validation, and test sets
full_split = dataset.train_test_split(test_size=0.2, seed=42)
train_val_split = full_split["train"].train_test_split(test_size=0.1, seed=42)

train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]
test_dataset = full_split["test"]

print(f"Train: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")

# Load GPT-2 tokenizer and model for sequence classification
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 does not have a default pad token

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=1024)

for entry in all_data:
    token_count = len(tokenizer.tokenize(entry["text"]))
    if token_count > 512:
        print(f"Text for grade {entry['label']} requires chunking. Token count: {token_count}")

# Apply tokenization
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set dataset format for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Train: 30031, Validation: 3337, Test: 8342


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Text for grade 2 requires chunking. Token count: 563
Text for grade 2 requires chunking. Token count: 563
Text for grade 2 requires chunking. Token count: 563
Text for grade 2 requires chunking. Token count: 563


Map:   0%|          | 0/30031 [00:00<?, ? examples/s]

Map:   0%|          | 0/3337 [00:00<?, ? examples/s]

Map:   0%|          | 0/8342 [00:00<?, ? examples/s]

In [ ]:
print(f"Total dataset entries: {len(all_data)}")

grade_counts = Counter(entry["label"] for entry in all_data)
print("Dataset entries per grade:", grade_counts)

# Check unique labels in the dataset
train_labels = set(train_dataset["label"])
val_labels = set(val_dataset["label"])
test_labels = set(test_dataset["label"])

print(f"Train labels: {train_labels}")
print(f"Validation labels: {val_labels}")
print(f"Test labels: {test_labels}")

# Output should only be {0, 1} for binary classification

Total dataset entries: 41710
Dataset entries per grade: Counter({2: 16101, 0: 12953, 1: 12656})
Train labels: {tensor(0), tensor(2), tensor(2), tensor(1), tensor(2), tensor(2), tensor(0), tensor(0), tensor(2), tensor(0), tensor(2), tensor(0), tensor(1), tensor(1), tensor(2), tensor(0), tensor(0), tensor(2), tensor(1), tensor(1), tensor(2), tensor(0), tensor(0), tensor(0), tensor(0), tensor(1), tensor(0), tensor(1), tensor(0), tensor(2), tensor(2), tensor(2), tensor(0), tensor(2), tensor(0), tensor(0), tensor(1), tensor(0), tensor(1), tensor(2), tensor(1), tensor(0), tensor(0), tensor(1), tensor(2), tensor(2), tensor(0), tensor(1), tensor(0), tensor(2), tensor(0), tensor(0), tensor(2), tensor(2), tensor(1), tensor(2), tensor(0), tensor(2), tensor(1), tensor(2), tensor(2), tensor(1), tensor(2), tensor(1), tensor(1), tensor(2), tensor(0), tensor(1), tensor(1), tensor(2), tensor(0), tensor(1), tensor(0), tensor(2), tensor(2), tensor(2), tensor(2), tensor(2), tensor(1), tensor(2), tensor(2)

In [ ]:
# Model with classification head (num_labels = total grade levels)
model = GPT2ForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=3  # Adjust based on the number of clusters or classes
)

# Set the pad_token_id in the model configuration
model.config.pad_token_id = model.config.eos_token_id

# Training arguments
training_args = TrainingArguments(
    output_dir="NLP_Final_Project/Model_Results_GPT2",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=12,
    weight_decay=0.01,
    logging_dir="NLP_Final_Project/Model_logs_GPT2",
    logging_steps=50,
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

metric = load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate on the validation set (during training)
val_results = trainer.evaluate(val_dataset)
print("Validation Results:", val_results)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


<ipython-input-9-cb8a45087310>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.618200,0.597448,0.708121
2,0.531100,0.560222,0.711118
3,0.507800,0.558564,0.731795
4,0.485200,0.544508,0.732394
5,0.428400,0.569545,0.729398
6,0.440800,0.574595,0.740485
7,0.421500,0.592125,0.736290
8,0.448500,0.573828,0.737489
9,0.410200,0.592873,0.750075
10,0.401800,0.601431,0.740485


Epoch,Training Loss,Validation Loss,Accuracy
1,0.618200,0.597448,0.708121
2,0.531100,0.560222,0.711118
3,0.507800,0.558564,0.731795
4,0.485200,0.544508,0.732394
5,0.428400,0.569545,0.729398
6,0.440800,0.574595,0.740485
7,0.421500,0.592125,0.736290
8,0.448500,0.573828,0.737489
9,0.410200,0.592873,0.750075
10,0.401800,0.601431,0.740485


Validation Results: {'eval_loss': 0.5928728580474854, 'eval_accuracy': 0.7500749175906503, 'eval_runtime': 49.455, 'eval_samples_per_second': 67.476, 'eval_steps_per_second': 4.226, 'epoch': 12.0}


In [ ]:
# Evaluate on the test set
test_results = trainer.evaluate(test_dataset)
print("Test Results:", test_results)

# Save the fine-tuned model
trainer.save_model("NLP_Final_Project/Fine_Tuned_Model_GPT2")

with open("test_results.json", "w") as f:
    json.dump(test_results, f)

Test Results: {'eval_loss': 0.5753780007362366, 'eval_accuracy': 0.7495804363462, 'eval_runtime': 123.438, 'eval_samples_per_second': 67.581, 'eval_steps_per_second': 4.229, 'epoch': 12.0}


In [ ]:
# Get predictions and labels
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

# Generate report
print(classification_report(labels, predictions, target_names=[str(i) for i in range(3)]))

              precision    recall  f1-score   support

           0       0.66      0.82      0.73      2479
           1       0.73      0.68      0.71      2583
           2       0.86      0.75      0.80      3280

    accuracy                           0.75      8342
   macro avg       0.75      0.75      0.75      8342
weighted avg       0.76      0.75      0.75      8342



In [ ]:
def chunk_text(text, max_tokens, tokenizer):
    """
    Split a long text into smaller chunks based on max_tokens.
    """
    tokens = tokenizer.tokenize(text)
    chunks = [
        tokenizer.convert_tokens_to_string(tokens[i : i + max_tokens])
        for i in range(0, len(tokens), max_tokens)
    ]
    return chunks


In [ ]:
# Updated function to include true grade and chunked texts
def extract_and_chunk_json(json_file, grade_mapping, tokenizer, max_tokens=1024):
    """
    Extract true grades and chunked texts from the JSON file.
    """
    curriculum_data = parse_json_with_multiple_grades(json_file)
    chunked_texts_by_grade = defaultdict(list)

    for entry in curriculum_data:
        true_grade = grade_mapping.get(entry["label"], -1)
        if true_grade == -1:
            continue  # Skip invalid grades

        # Chunk text
        chunks = chunk_text(entry["text"], max_tokens, tokenizer)

        # Append each chunk with a suffix
        for i, chunk in enumerate(chunks):
            chunked_texts_by_grade[true_grade].append(f"Grade_{true_grade}_{i + 1}: {chunk}")

    return chunked_texts_by_grade

In [ ]:
def predict_chunks(trainer, dataset):
    """
    Predict clusters for all chunks in the dataset.
    """
    # Predict logits for each chunk
    logits = trainer.predict(dataset).predictions
    probabilities = torch.softmax(torch.tensor(logits), dim=-1).numpy()

    # Aggregate predictions by grade
    aggregated_results = defaultdict(list)
    for idx, grade in enumerate(dataset["grade"]):
        aggregated_results[int(grade)].append(probabilities[idx])  # Ensure grade is int

    # Compute average probabilities and determine the predicted cluster
    final_results = {}
    for grade, grade_probs in aggregated_results.items():
        avg_probs = np.mean(grade_probs, axis=0)
        predicted_cluster = np.argmax(avg_probs)

        final_results[grade] = {
            "predicted_cluster": predicted_cluster,
            "chunk_probabilities": grade_probs,
            "average_probabilities": avg_probs,
        }

        print(f"Probabilities for grade {grade}: {grade_probs}")
        print(f"Average probabilities for grade {grade}: {avg_probs}")

    return final_results

In [ ]:
def analyze_results_with_chunks(evaluation_results):
    """
    Compute metrics for binary classification and cluster correction for chunked data.
    """
    binary_metrics = defaultdict(list)
    cluster_metrics = defaultdict(list)

    for true_grade, result in evaluation_results.items():
        # Convert true_grade to int if not already
        true_grade = int(true_grade)

        # Get true cluster and predicted cluster
        true_cluster = grade_cluster_mapping.get(true_grade, -1)
        predicted_cluster = result.get("predicted_cluster")

        if true_cluster == -1 or predicted_cluster is None:
            print(f"Skipping invalid grade {true_grade}: true_cluster={true_cluster}, predicted_cluster={predicted_cluster}")
            continue

        # Binary classification metrics
        binary_label = 1 if true_cluster == predicted_cluster else 0
        binary_metrics["true"].append(1)  # All grades are inherently "Appropriate"
        binary_metrics["predicted"].append(binary_label)

        # Cluster correction metrics
        cluster_metrics["true"].append(true_cluster)
        cluster_metrics["predicted"].append(predicted_cluster)

        # Debug: Print probabilities for each grade
        print(f"True Grade: {true_grade}, Predicted Cluster: {predicted_cluster}")
        print(f"Average Probabilities: {result['average_probabilities']}")

    # Compute final metrics
    binary_accuracy = accuracy_score(binary_metrics["true"], binary_metrics["predicted"])
    binary_precision, binary_recall, binary_f1, _ = precision_recall_fscore_support(
        binary_metrics["true"], binary_metrics["predicted"], average="binary"
    )
    cluster_accuracy = accuracy_score(cluster_metrics["true"], cluster_metrics["predicted"])
    cluster_precision, cluster_recall, cluster_f1, _ = precision_recall_fscore_support(
        cluster_metrics["true"], cluster_metrics["predicted"], average="macro"
    )

    print(f"Binary Metrics True Labels: {binary_metrics['true']}")
    print(f"Binary Metrics Predicted Labels: {binary_metrics['predicted']}")

    return {
        "binary_classification": {
            "accuracy": binary_accuracy,
            "precision": binary_precision,
            "recall": binary_recall,
            "f1_score": binary_f1,
        },
        "cluster_correction": {
            "accuracy": cluster_accuracy,
            "precision": cluster_precision,
            "recall": cluster_recall,
            "f1_score": cluster_f1,
        },
    }

In [ ]:
# Preprocess JSON
chunked_texts = extract_and_chunk_json(
    json_file="NLP_Final_Project/State_JSON/New_York_Standards.json",
    grade_mapping=grade_mapping,
    tokenizer=tokenizer,
    max_tokens=1024
)

# Prepare a dataset that includes grades
texts, grades = [], []
for grade, chunks in chunked_texts.items():
    texts.extend(chunks)
    grades.extend([grade] * len(chunks))

# Create Dataset with text and grades
dataset = Dataset.from_dict({"text": texts, "grade": grades})
dataset = dataset.map(tokenize, batched=True)
dataset.set_format("torch", columns=["input_ids", "attention_mask", "grade"])

Map:   0%|          | 0/2690 [00:00<?, ? examples/s]

'print("Binary Classification Metrics:")\nfor metric, value in metrics["binary_classification"].items():\n    print(f"{metric.capitalize()}: {value:.4f}")\n\nprint("\nCluster Correction Metrics:")\nfor metric, value in metrics["cluster_correction"].items():\n    print(f"{metric.capitalize()}: {value:.4f}")'

In [ ]:
# Baseline: Curriculum-based
grade_cluster_mapping_baseline = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 2, 10: 2, 11: 2, 12: 2}

# Practical grouping
grade_cluster_mapping_practical = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 1, 9: 2, 10: 2, 11: 2, 12: 2}

# Teaching pairs
grade_cluster_mapping_pairs = {
    0: 0, 1: 0, 2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 3, 8: 4, 9: 4, 10: 5, 11: 5, 12: 5
}

# Teaching triple
grade_cluster_mapping_triple = {
    0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1, 6: 2, 7: 2, 8: 2, 9: 3, 10: 3, 11: 3, 12: 3
}

# Individual grades
grade_cluster_mapping_individual = {i: i for i in range(13)}

# Replace `grade_cluster_mapping` dynamically
cluster_schemes = {
    "baseline": grade_cluster_mapping_baseline,
    "practical": grade_cluster_mapping_practical,
    "pairs": grade_cluster_mapping_pairs,
    "triple": grade_cluster_mapping_triple,
    "individual": grade_cluster_mapping_individual,
}

results = {}
for scheme_name, mapping in cluster_schemes.items():
    grade_cluster_mapping = mapping
    evaluation_results = predict_chunks(trainer, dataset)
    metrics = analyze_results_with_chunks(evaluation_results)
    results[scheme_name] = metrics
    print(f"Results for {scheme_name}:")
    print(metrics)

Probabilities for grade 0: [array([0.5084052 , 0.24432729, 0.24726751], dtype=float32), array([0.5458571 , 0.3501007 , 0.10404222], dtype=float32), array([0.6022113 , 0.32611385, 0.07167486], dtype=float32), array([0.70713323, 0.26582986, 0.02703683], dtype=float32), array([0.36119178, 0.3586365 , 0.28017178], dtype=float32), array([0.42936316, 0.3452052 , 0.22543165], dtype=float32), array([0.42130685, 0.30445004, 0.27424312], dtype=float32), array([0.42843896, 0.40151027, 0.17005076], dtype=float32), array([0.3718103 , 0.35866734, 0.26952237], dtype=float32), array([0.42961818, 0.31620377, 0.25417802], dtype=float32), array([0.38865215, 0.33008444, 0.2812634 ], dtype=float32), array([0.05780501, 0.46161252, 0.48058245], dtype=float32), array([0.33870918, 0.40364662, 0.2576442 ], dtype=float32), array([0.48286483, 0.3281652 , 0.18896995], dtype=float32), array([0.43229106, 0.3722167 , 0.19549228], dtype=float32), array([0.46251252, 0.32673347, 0.21075398], dtype=float32), array([0.372

Probabilities for grade 0: [array([0.5084052 , 0.24432729, 0.24726751], dtype=float32), array([0.5458571 , 0.3501007 , 0.10404222], dtype=float32), array([0.6022113 , 0.32611385, 0.07167486], dtype=float32), array([0.70713323, 0.26582986, 0.02703683], dtype=float32), array([0.36119178, 0.3586365 , 0.28017178], dtype=float32), array([0.42936316, 0.3452052 , 0.22543165], dtype=float32), array([0.42130685, 0.30445004, 0.27424312], dtype=float32), array([0.42843896, 0.40151027, 0.17005076], dtype=float32), array([0.3718103 , 0.35866734, 0.26952237], dtype=float32), array([0.42961818, 0.31620377, 0.25417802], dtype=float32), array([0.38865215, 0.33008444, 0.2812634 ], dtype=float32), array([0.05780501, 0.46161252, 0.48058245], dtype=float32), array([0.33870918, 0.40364662, 0.2576442 ], dtype=float32), array([0.48286483, 0.3281652 , 0.18896995], dtype=float32), array([0.43229106, 0.3722167 , 0.19549228], dtype=float32), array([0.46251252, 0.32673347, 0.21075398], dtype=float32), array([0.372

Probabilities for grade 0: [array([0.5084052 , 0.24432729, 0.24726751], dtype=float32), array([0.5458571 , 0.3501007 , 0.10404222], dtype=float32), array([0.6022113 , 0.32611385, 0.07167486], dtype=float32), array([0.70713323, 0.26582986, 0.02703683], dtype=float32), array([0.36119178, 0.3586365 , 0.28017178], dtype=float32), array([0.42936316, 0.3452052 , 0.22543165], dtype=float32), array([0.42130685, 0.30445004, 0.27424312], dtype=float32), array([0.42843896, 0.40151027, 0.17005076], dtype=float32), array([0.3718103 , 0.35866734, 0.26952237], dtype=float32), array([0.42961818, 0.31620377, 0.25417802], dtype=float32), array([0.38865215, 0.33008444, 0.2812634 ], dtype=float32), array([0.05780501, 0.46161252, 0.48058245], dtype=float32), array([0.33870918, 0.40364662, 0.2576442 ], dtype=float32), array([0.48286483, 0.3281652 , 0.18896995], dtype=float32), array([0.43229106, 0.3722167 , 0.19549228], dtype=float32), array([0.46251252, 0.32673347, 0.21075398], dtype=float32), array([0.372

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Probabilities for grade 0: [array([0.5084052 , 0.24432729, 0.24726751], dtype=float32), array([0.5458571 , 0.3501007 , 0.10404222], dtype=float32), array([0.6022113 , 0.32611385, 0.07167486], dtype=float32), array([0.70713323, 0.26582986, 0.02703683], dtype=float32), array([0.36119178, 0.3586365 , 0.28017178], dtype=float32), array([0.42936316, 0.3452052 , 0.22543165], dtype=float32), array([0.42130685, 0.30445004, 0.27424312], dtype=float32), array([0.42843896, 0.40151027, 0.17005076], dtype=float32), array([0.3718103 , 0.35866734, 0.26952237], dtype=float32), array([0.42961818, 0.31620377, 0.25417802], dtype=float32), array([0.38865215, 0.33008444, 0.2812634 ], dtype=float32), array([0.05780501, 0.46161252, 0.48058245], dtype=float32), array([0.33870918, 0.40364662, 0.2576442 ], dtype=float32), array([0.48286483, 0.3281652 , 0.18896995], dtype=float32), array([0.43229106, 0.3722167 , 0.19549228], dtype=float32), array([0.46251252, 0.32673347, 0.21075398], dtype=float32), array([0.372

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Probabilities for grade 0: [array([0.5084052 , 0.24432729, 0.24726751], dtype=float32), array([0.5458571 , 0.3501007 , 0.10404222], dtype=float32), array([0.6022113 , 0.32611385, 0.07167486], dtype=float32), array([0.70713323, 0.26582986, 0.02703683], dtype=float32), array([0.36119178, 0.3586365 , 0.28017178], dtype=float32), array([0.42936316, 0.3452052 , 0.22543165], dtype=float32), array([0.42130685, 0.30445004, 0.27424312], dtype=float32), array([0.42843896, 0.40151027, 0.17005076], dtype=float32), array([0.3718103 , 0.35866734, 0.26952237], dtype=float32), array([0.42961818, 0.31620377, 0.25417802], dtype=float32), array([0.38865215, 0.33008444, 0.2812634 ], dtype=float32), array([0.05780501, 0.46161252, 0.48058245], dtype=float32), array([0.33870918, 0.40364662, 0.2576442 ], dtype=float32), array([0.48286483, 0.3281652 , 0.18896995], dtype=float32), array([0.43229106, 0.3722167 , 0.19549228], dtype=float32), array([0.46251252, 0.32673347, 0.21075398], dtype=float32), array([0.372

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print(metrics)

print(f"Evaluation Results Keys: {list(evaluation_results.keys())}")
print(f"Sample Evaluation Result: {next(iter(evaluation_results.items())) if evaluation_results else 'No data'}")


{'binary_classification': {'accuracy': 0.07692307692307693, 'precision': 1.0, 'recall': 0.07692307692307693, 'f1_score': 0.14285714285714285}, 'cluster_correction': {'accuracy': 0.07692307692307693, 'precision': 0.015384615384615385, 'recall': 0.07692307692307693, 'f1_score': 0.02564102564102564}}
Evaluation Results Keys: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Sample Evaluation Result: (0, {'predicted_cluster': 0, 'chunk_probabilities': [array([0.5084052 , 0.24432729, 0.24726751], dtype=float32), array([0.5458571 , 0.3501007 , 0.10404222], dtype=float32), array([0.6022113 , 0.32611385, 0.07167486], dtype=float32), array([0.70713323, 0.26582986, 0.02703683], dtype=float32), array([0.36119178, 0.3586365 , 0.28017178], dtype=float32), array([0.42936316, 0.3452052 , 0.22543165], dtype=float32), array([0.42130685, 0.30445004, 0.27424312], dtype=float32), array([0.42843896, 0.40151027, 0.17005076], dtype=float32), array([0.3718103 , 0.35866734, 0.26952237], dtype=float32), array([0.42961

In [ ]:
# Debugging aggregated results
for grade, result in evaluation_results.items():
    print(f"True Grade: {grade}")
    print(f"Predicted Cluster: {result['predicted_cluster']}")
    print(f"Chunk Probabilities: {result['chunk_probabilities']}")
    print(f"Average Probabilities: {result['average_probabilities']}")


True Grade: 0
Predicted Cluster: 0
Chunk Probabilities: [array([0.5084052 , 0.24432729, 0.24726751], dtype=float32), array([0.5458571 , 0.3501007 , 0.10404222], dtype=float32), array([0.6022113 , 0.32611385, 0.07167486], dtype=float32), array([0.70713323, 0.26582986, 0.02703683], dtype=float32), array([0.36119178, 0.3586365 , 0.28017178], dtype=float32), array([0.42936316, 0.3452052 , 0.22543165], dtype=float32), array([0.42130685, 0.30445004, 0.27424312], dtype=float32), array([0.42843896, 0.40151027, 0.17005076], dtype=float32), array([0.3718103 , 0.35866734, 0.26952237], dtype=float32), array([0.42961818, 0.31620377, 0.25417802], dtype=float32), array([0.38865215, 0.33008444, 0.2812634 ], dtype=float32), array([0.05780501, 0.46161252, 0.48058245], dtype=float32), array([0.33870918, 0.40364662, 0.2576442 ], dtype=float32), array([0.48286483, 0.3281652 , 0.18896995], dtype=float32), array([0.43229106, 0.3722167 , 0.19549228], dtype=float32), array([0.46251252, 0.32673347, 0.21075398],

In [ ]:
for grade in evaluation_results.keys():
    if grade not in grade_cluster_mapping:
        print(f"Warning: Grade {grade} not found in grade_cluster_mapping.")

for grade, result in evaluation_results.items():
    predicted_cluster = result['predicted_cluster']
    avg_probs = result['average_probabilities']
    max_prob_cluster = np.argmax(avg_probs)
    if predicted_cluster != max_prob_cluster:
        print(f"Inconsistent Prediction for Grade {grade}: Predicted={predicted_cluster}, Max Prob={max_prob_cluster}")